In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.models import Sequential
from transformers import  BertTokenizer, TFBertForSequenceClassification, TFDistilBertForSequenceClassification, DistilBertTokenizer

KeyboardInterrupt: 

In [ ]:
df= pd.read_csv(r'dataset\balance.csv')
df.head()

,Rating,Description
0,5,Klenteng Ban Hin Kiong merupakan Klenteng tert...
1,5,Airnya sejuk. Tempatnya bebas plastik. Bagi ya...
2,5,Tiap minggu pasti kesini buat foto2 doang😁 kar...
3,5,"Pernah kesana pergi liat bunker jepang, naik p..."
4,5,"Mengikuti Talkshow "" Menyingkap Pesona Wastra ..."


In [ ]:
len(df.Rating)

766

In [ ]:
map_label = {1 : 0,
             2 : 0,
             3: 1,
             4 : 2,
             5 : 2}
df['Rating'] = df['Rating'].map(map_label)

In [ ]:
df['Description'] = df['Description'].astype(str)
df['Description'] = df['Description'].str.lower()

In [ ]:
# memisahkan text dan label

text = df['Description'].tolist()
label = df['Rating'].tolist()

In [ ]:
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-large-p1')

Downloading: 100%|██████████| 229k/229k [00:00<00:00, 467kB/s] 
Downloading: 100%|██████████| 112/112 [00:00<?, ?B/s] 
Downloading: 100%|██████████| 2.00/2.00 [00:00<00:00, 2.01kB/s]


In [ ]:
def tokenize_data(texts, labels, tokenizer, max_length=128):
    input_ids = []
    attention_mask = []
    for text in texts:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True
        )
        input_ids.append(encoded['input_ids'])
        attention_mask.append(encoded['attention_mask'])

    return np.array(input_ids), np.array(attention_mask), np.array(labels)

In [ ]:
# Preprocessing: Tokenisasi dan padding
max_length = 128
input_ids, attention_mask, labels = tokenize_data(text, label, tokenizer, max_length)

In [ ]:
train_inputs, test_inputs, train_labels, test_labels = train_test_split(input_ids, labels, test_size=0.2, random_state=42)
train_masks, test_masks, _, _ = train_test_split(attention_mask, labels, test_size=0.2, random_state=42)

In [ ]:
# Mengubah menjadi tensordata
train_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': train_inputs, 'attention_mask': train_masks}, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': test_inputs, 'attention_mask': test_masks}, test_labels))

In [ ]:
# Batching data
batch_size = 4
train_dataset = train_dataset.shuffle(100).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

muat model indobert

In [ ]:
model = TFBertForSequenceClassification.from_pretrained('indobenchmark/indobert-large-p1', num_labels=3)

Downloading: 100%|██████████| 1.53k/1.53k [00:00<00:00, 1.54MB/s]
Downloading: 100%|██████████| 1.47G/1.47G [04:08<00:00, 5.93MB/s]
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-large-p1 and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


compile model

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-8)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=tf.keras.metrics.SparseCategoricalAccuracy('accuracy'))

In [ ]:
# Latih model

history = model.fit(train_dataset,
                    epochs=4,
                    validation_data=test_dataset)

Epoch 1/4
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


153/153 [==============================] - ETA: 0s - loss: 0.9029 - accuracy: 0.6225

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


153/153 [==============================] - 96s 368ms/step - loss: 0.9029 - accuracy: 0.6225 - val_loss: 0.8976 - val_accuracy: 0.5714
Epoch 2/4
153/153 [==============================] - 50s 324ms/step - loss: 0.6494 - accuracy: 0.7451 - val_loss: 0.8281 - val_accuracy: 0.6039
Epoch 3/4
153/153 [==============================] - 49s 323ms/step - loss: 0.4352 - accuracy: 0.8301 - val_loss: 0.7853 - val_accuracy: 0.6818
Epoch 4/4
153/153 [==============================] - 50s 329ms/step - loss: 0.2657 - accuracy: 0.9101 - val_loss: 0.8739 - val_accuracy: 0.6623
